# SMS 相似性配對 - LLM 輔助對比學習正樣本生成

此 notebook 使用 LLM (Together AI) 來找出簡訊數據中兩兩相似的內容，為對比學習生成正樣本對。

## 流程概述
1. 載入 SMS 數據
2. 使用 LLM 比較簡訊相似性
3. 生成相似簡訊對的 CSV 檔案

In [1]:
import pandas as pd
import numpy as np
from together import Together
from tqdm import tqdm
import time
import os
import json
from typing import List, Tuple
from itertools import combinations
import textwrap

In [2]:
# 設置 Together AI API 客戶端
# 讀取 API keys
try:
    with open("data_game/together_api_key.txt", "r") as f:
        api_keys = [line.strip() for line in f.readlines() if line.strip()]
except FileNotFoundError:
    print("請確保 data_game/together_api_key.txt 文件存在")
    raise

# 確保有足夠的 API key
if len(api_keys) < 4:
    raise ValueError("請在 together_api_key.txt 文件中提供四個 API key，每行一個")

# 創建四個 Together 客戶端進行負載均衡
together_clients = [Together(api_key=key) for key in api_keys]

# 追蹤當前使用哪個客戶端
_current_client_index = 0

def get_current_together_client():
    """循環返回四個 Together 客戶端"""
    global _current_client_index
    current_client = together_clients[_current_client_index]
    _current_client_index = (_current_client_index + 1) % len(together_clients)
    return current_client

print(f"已成功初始化 {len(together_clients)} 個 Together AI 客戶端")
for i, key in enumerate(api_keys):
    print(f"API Key {i+1}: {key[:10]}...")

已成功初始化 4 個 Together AI 客戶端
API Key 1: 1f3b522748...
API Key 2: 542d70c17d...
API Key 3: 713f17cd14...
API Key 4: db7765986a...


In [3]:
# 載入 SMS 數據並分析 label 分布
df = pd.read_csv("datagame_sms_stage1.csv")
print(f"載入了 {len(df)} 筆簡訊數據")
print("\n數據結構:")
print(df.head())
print(f"\n欄位: {list(df.columns)}")

# 只保留有簡訊內容的數據
df = df.dropna(subset=['sms_body'])
print(f"\n過濾後剩餘 {len(df)} 筆有效簡訊")

# 分析 label 分布
print(f"\n=== Label 分布分析 ===")
label_counts = df['label'].value_counts()
print(label_counts)

# 分離旅遊類型和非旅遊類型簡訊
travel_sms = df[df['label'] == 1].copy()
non_travel_sms = df[df['label'] != 1].copy()

print(f"\n旅遊類型簡訊 (label=1): {len(travel_sms)} 筆")
print(f"非旅遊類型簡訊 (label≠1): {len(non_travel_sms)} 筆")

# 顯示一些樣本
print(f"\n=== 旅遊類型簡訊樣本 ===")
for i, row in travel_sms.head(3).iterrows():
    print(f"ID {row['sms_id']}: {row['sms_body']}")

print(f"\n=== 非旅遊類型簡訊樣本 ===")
for i, row in non_travel_sms.head(3).iterrows():
    print(f"ID {row['sms_id']}: {row['sms_body']}")

載入了 209481 筆簡訊數據

數據結構:
   sms_id                                           sms_body  label  name_flg
0  162569  親愛的家長您好，寶寶即將滿一歲，記得安排回兒科施打麻疹腮腺炎德國麻疹(MMR)疫苗，並攜帶健...    NaN       NaN
1  314614  富邦帳戶轉帳完成：您於05/28上午10:14轉出NT$4,200元至王道銀行帳戶末四碼71...    NaN       NaN
2  355174  【測驗通知】張睿庭學員：本週五為「自然科探究與實作能力評量」，共90分鐘，請攜帶實驗筆記與護...    NaN       NaN
3   28258  詹睿哲君，您好：提醒您5月分手機分期款NT$2,100至今尚未入帳，系統已發送3次通知未回覆...    NaN       NaN
4  376466  尊敬的客戶，這是來自中信銀行的提醒。您的貸款款項已過期，請您儘速償還。若已繳款，請無需理會此...    NaN       NaN

欄位: ['sms_id', 'sms_body', 'label', 'name_flg']

過濾後剩餘 209481 筆有效簡訊

=== Label 分布分析 ===
label
1.0    1000
Name: count, dtype: int64

旅遊類型簡訊 (label=1): 1000 筆
非旅遊類型簡訊 (label≠1): 208481 筆

=== 旅遊類型簡訊樣本 ===
ID 254284: 來場異國文化之旅！「異域風情」旅行社推出的七日印度之旅，帶您探索德里、阿格拉等地，感受印度的歷史與文化底蘊。報名即享早鳥優惠，立刻報名，開啟這場精彩的印度之旅！報名熱線：02-23335588。
ID 229496: 李麗芬小姐，您的「日本沖繩海島之旅」已確認，所有行程與住宿將於出發前三天發送至您的電子信箱。
ID 44913: 👨‍👩‍👧王子維與林郁茹報名參加「北海道雪地溫泉假期」，李宥帆也強烈推薦滑雪課程＋夜間泡湯體驗，限時早鳥現折$2000！

=== 非旅遊類型簡訊樣本 ===
ID 162569: 親愛的家長您好，寶寶即將滿一歲，記得安排回兒科施打麻疹腮腺炎德國麻疹(

In [ ]:
# 限流設置
RPM = 60                # requests/min
WINDOW = 60             # sec
MAX_RETRY = 3           # retries
MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

# 請求紀錄
_req_log = []

def _throttle():
    """簡單的限流機制"""
    now = time.time()
    # 清理超過時間窗口的記錄
    _req_log[:] = [t for t in _req_log if now - t < WINDOW]
    
    # 檢查是否需要等待
    if len(_req_log) >= RPM:
        wait_time = WINDOW - (now - _req_log[0])
        if wait_time > 0:
            time.sleep(wait_time)
    
    _req_log.append(now)

# 設計批量相似性判斷的 prompt
batch_similarity_prompt_template = textwrap.dedent(
    """
    你是一個專業的文本相似性判斷助手。請仔細比較以下3個簡訊的內容，找出其中相似的簡訊對。
    判斷標準:
    1. 主題相似性：兩個簡訊是否討論相同或相關的主題
    2. 意圖相似性：兩個簡訊的目的是否相似（如：通知、推廣、提醒等）
    3. 內容相似性：具體內容是否有重疊或相關性
    注意：
    - 即使措詞不同，但如果主題和意圖相似，也應判斷為相似
    - 只是格式相似但內容完全不同的簡訊不應判斷為相似
    - 考慮簡訊的實際含義而非表面文字
    請按照以下格式回答，每行一個比較結果：
    A-B: 1 (如果簡訊A和B相似) 或 0 (如果不相似)
    A-C: 1 (如果簡訊A和C相似) 或 0 (如果不相似)  
    B-C: 1 (如果簡訊B和C相似) 或 0 (如果不相似)
    簡訊A: {sms_a}
    簡訊B: {sms_b}
    簡訊C: {sms_c}
    """
)

def check_batch_sms_similarity(sms_list: List[Tuple[int, str]]) -> List[Tuple[int, int]]:
    """
    使用 LLM 批量判斷3個簡訊的相似性
    
    Args:
        sms_list: [(sms_id, sms_body), ...] 最多3個簡訊
    
    Returns:
        相似簡訊對的列表 [(id1, id2), ...]
    """
    if len(sms_list) != 3:
        raise ValueError("批量比較需要恰好3個簡訊")
    
    (id_a, sms_a), (id_b, sms_b), (id_c, sms_c) = sms_list
    
    prompt = batch_similarity_prompt_template.format(
        sms_a=sms_a, 
        sms_b=sms_b, 
        sms_c=sms_c
    )
    
    client = get_current_together_client()
    
    for attempt in range(MAX_RETRY):
        try:
            _throttle()
            response = client.chat.completions.create(
                model=MODEL_ID,
                messages=[
                    {"role": "user", "content": prompt}
                ],
                max_tokens=20,  # 增加token數量以應對多行輸出
                temperature=0.0,
                stream=False,
            )
            
            result = response.choices[0].message.content.strip()
            
            # 解析結果
            similar_pairs = []
            lines = result.split('\n')
            
            # 解析格式：A-B: 1, A-C: 0, B-C: 1
            for line in lines:
                line = line.strip()
                if ':' in line:
                    pair_part, similarity_part = line.split(':', 1)
                    pair_part = pair_part.strip()
                    similarity = similarity_part.strip()
                    
                    if similarity == '1':
                        if pair_part.upper() == 'A-B':
                            similar_pairs.append((id_a, id_b))
                        elif pair_part.upper() == 'A-C':
                            similar_pairs.append((id_a, id_c))
                        elif pair_part.upper() == 'B-C':
                            similar_pairs.append((id_b, id_c))
            
            return similar_pairs
                
        except Exception as e:
            if attempt + 1 >= MAX_RETRY:
                print(f"Error after {MAX_RETRY} attempts: {e}")
                print(f"Raw response: {result if 'result' in locals() else 'No response'}")
                return []  # 失敗時返回空列表
            
            print(f"Attempt {attempt + 1} failed: {e}. Retrying in 60 seconds...")
            time.sleep(60)
    
    return []

# 保留原始單對比較函數作為備用
def check_sms_similarity(sms1: str, sms2: str) -> str:
    """使用 LLM 判斷兩個簡訊是否相似（單對比較）"""
    similarity_prompt_template = textwrap.dedent(
        """
        你是一個專業的文本相似性判斷助手。請仔細比較以下兩個簡訊的內容，判斷它們是否在語義上相似。

        判斷標準:
        1. 主題相似性：兩個簡訊是否討論相同或相關的主題
        2. 意圖相似性：兩個簡訊的目的是否相似（如：通知、推廣、提醒等）
        3. 內容相似性：具體內容是否有重疊或相關性
        
        注意：
        - 即使措詞不同，但如果主題和意圖相似，也應判斷為相似
        - 只是格式相似但內容完全不同的簡訊不應判斷為相似
        - 考慮簡訊的實際含義而非表面文字

        請只回答 "1"（相似）或 "0"（不相似），不要輸出其他內容。

        簡訊1: {sms1}
        
        簡訊2: {sms2}
        """
    )
    
    prompt = similarity_prompt_template.format(sms1=sms1, sms2=sms2)
    
    client = get_current_together_client()
    
    for attempt in range(MAX_RETRY):
        try:
            _throttle()
            response = client.chat.completions.create(
                model=MODEL_ID,
                messages=[
                    {"role": "user", "content": prompt}
                ],
                max_tokens=4,
                temperature=0.0,
                stream=False,
            )
            
            result = response.choices[0].message.content.strip()
            if result in ["0", "1"]:
                return result
            else:
                print(f"Warning: Unexpected output '{result}', treating as not similar")
                return "0"
                
        except Exception as e:
            if attempt + 1 >= MAX_RETRY:
                print(f"Error after {MAX_RETRY} attempts: {e}")
                return "0"  # 失敗時預設為不相似
            
            print(f"Attempt {attempt + 1} failed: {e}. Retrying in 60 seconds...")
            time.sleep(60)
    
    return "0"

print("批量相似性判斷功能已設置完成")

批量相似性判斷功能已設置完成


In [5]:
# 批量同類型簡訊配對函數
def find_same_type_pairs_batch(df_subset: pd.DataFrame, max_pairs: int = None, description: str = "") -> List[Tuple[int, int]]:
    """
    在同類型簡訊中使用批量比較找出相似對
    
    Args:
        df_subset: 同類型的簡訊 DataFrame
        max_pairs: 最大要找的相似對數量
        description: 描述文字
    
    Returns:
        相似簡訊對的列表 [(id1, id2), ...]
    """
    if len(df_subset) < 3:
        print(f"{description}: 數據量不足（需要至少3筆），改用單對比較")
        return find_same_type_pairs_single(df_subset, max_pairs, description)
    
    similar_pairs = []
    df_list = list(df_subset.iterrows())
    
    if max_pairs is None:
        max_pairs = min(len(df_subset) * (len(df_subset) - 1) // 2, 1000)  # 預設最多1000對
    
    print(f"\n=== {description} (批量模式) ===")
    print(f"將批量比較 {len(df_subset)} 筆簡訊")
    print(f"目標找到 {max_pairs} 個相似對")
    
    # 計算需要的批次數量（每3個簡訊為一批）
    total_batches = len(df_list) // 3
    if len(df_list) % 3 != 0:
        total_batches += 1
    
    # 使用 tqdm 顯示進度
    with tqdm(total=min(total_batches, max_pairs), desc=f"{description}批量比較進度") as pbar:
        batch_count = 0
        
        # 批量處理：每3個簡訊為一組
        for i in range(0, len(df_list), 3):
            if len(similar_pairs) >= max_pairs:
                break
            
            # 取3個簡訊進行批量比較
            batch = df_list[i:i+3]
            if len(batch) == 3:
                # 準備批量比較的數據
                sms_batch = [(row['sms_id'], row['sms_body']) for idx, row in batch]
                
                # 進行批量相似性判斷
                batch_pairs = check_batch_sms_similarity(sms_batch)
                
                # 添加找到的相似對
                for pair in batch_pairs:
                    if len(similar_pairs) < max_pairs:
                        similar_pairs.append(pair)
                        
                        # 找到並顯示簡訊內容
                        id1, id2 = pair
                        sms1 = df_subset[df_subset['sms_id'] == id1]['sms_body'].iloc[0]
                        sms2 = df_subset[df_subset['sms_id'] == id2]['sms_body'].iloc[0]

                
                batch_count += 1
                pbar.update(1)
            
            # 處理剩餘的1-2個簡訊（如果有的話）
            elif len(batch) == 2 and len(similar_pairs) < max_pairs:
                # 對剩餘的2個簡訊進行單對比較
                row1, row2 = batch[0][1], batch[1][1]
                is_similar = check_sms_similarity(row1['sms_body'], row2['sms_body'])
                
                if is_similar == "1":
                    similar_pairs.append((row1['sms_id'], row2['sms_id']))
                
        
        # 如果還需要更多相似對，繼續進行交叉批量比較
        if len(similar_pairs) < max_pairs and len(df_list) >= 6:
            print(f"\n進行交叉批量比較以找到更多相似對...")
            cross_batch_count = 0
            max_cross_batches = min(50, max_pairs - len(similar_pairs))  # 限制交叉比較次數
            
            for i in range(0, len(df_list)-3, 6):  # 每6個簡訊取兩組進行交叉比較
                if len(similar_pairs) >= max_pairs or cross_batch_count >= max_cross_batches:
                    break
                
                # 取第一組3個簡訊
                batch1 = df_list[i:i+3]
                # 取第二組3個簡訊
                batch2 = df_list[i+3:i+6] if i+6 <= len(df_list) else df_list[i+3:]
                
                if len(batch1) == 3 and len(batch2) >= 2:
                    # 創建混合批次進行比較
                    for j in range(len(batch2)):
                        if len(similar_pairs) >= max_pairs:
                            break
                        
                        # 取batch1的前2個 + batch2的第j個組成新的批次
                        mixed_batch = [batch1[0], batch1[1], batch2[j]]
                        sms_batch = [(row['sms_id'], row['sms_body']) for idx, row in mixed_batch]
                        
                        batch_pairs = check_batch_sms_similarity(sms_batch)
                        
                        for pair in batch_pairs:
                            if len(similar_pairs) < max_pairs:
                                # 檢查是否為新的相似對
                                if pair not in similar_pairs and (pair[1], pair[0]) not in similar_pairs:
                                    similar_pairs.append(pair)
                                    
                                    id1, id2 = pair
                                    sms1 = df_subset[df_subset['sms_id'] == id1]['sms_body'].iloc[0]
                                    sms2 = df_subset[df_subset['sms_id'] == id2]['sms_body'].iloc[0]
                                    
                cross_batch_count += 1
                pbar.update(1)
    
    print(f"\n{description}完成！總共找到 {len(similar_pairs)} 個相似對")
    return similar_pairs

# 單對比較函數（備用）
def find_same_type_pairs_single(df_subset: pd.DataFrame, max_pairs: int = None, description: str = "") -> List[Tuple[int, int]]:
    """
    在同類型簡訊中使用單對比較找出相似對（備用方法）
    """
    if len(df_subset) < 2:
        print(f"{description}: 數據量不足，無法配對")
        return []
    
    similar_pairs = []
    total_combinations = len(df_subset) * (len(df_subset) - 1) // 2
    
    if max_pairs is None:
        max_pairs = min(total_combinations, 1000)
    
    print(f"\n=== {description} (單對模式) ===")
    print(f"將比較 {len(df_subset)} 筆簡訊，總共 {total_combinations} 個組合")
    print(f"目標找到 {max_pairs} 個相似對")
    
    with tqdm(total=min(total_combinations, max_pairs * 5), desc=f"{description}比較進度") as pbar:
        compared = 0
        
        for i, row1 in df_subset.iterrows():
            if len(similar_pairs) >= max_pairs:
                break
                
            for j, row2 in df_subset.iterrows():
                if i >= j:
                    continue
                
                compared += 1
                pbar.update(1)
                
                if len(similar_pairs) >= max_pairs:
                    break
                
                if compared > max_pairs * 5:
                    break
                
                is_similar = check_sms_similarity(row1['sms_body'], row2['sms_body'])
                
                if is_similar == "1":
                    similar_pairs.append((row1['sms_id'], row2['sms_id']))
                    print(f"\n找到第 {len(similar_pairs)} 個{description}相似對:")
                    print(f"ID {row1['sms_id']}: {row1['sms_body'][:50]}...")
                    print(f"ID {row2['sms_id']}: {row2['sms_body'][:50]}...")
                    
            if len(similar_pairs) >= max_pairs or compared > max_pairs * 5:
                break
    
    print(f"\n{description}完成！總共找到 {len(similar_pairs)} 個相似對")
    return similar_pairs

# 設置預設使用批量比較
find_same_type_pairs = find_same_type_pairs_batch

print("批量同類型配對功能已設置完成")

批量同類型配對功能已設置完成


In [ ]:
# 執行分類配對策略
print("開始執行分類配對策略...")

# ========== 第一階段：旅遊類型簡訊配對 ==========
print(f"\n🏖️ 第一階段：旅遊類型簡訊配對")
travel_pairs = find_same_type_pairs(
    df_subset=travel_sms,
    max_pairs=400,  # 盡量找出所有旅遊類型的相似對
    description="旅遊類型簡訊"
)

# ========== 第二階段：非旅遊類型簡訊配對 ==========
print(f"\n🏢 第二階段：非旅遊類型簡訊配對")

# 從非旅遊類型中採樣800筆
NON_TRAVEL_SAMPLE_SIZE = 40000
if len(non_travel_sms) > NON_TRAVEL_SAMPLE_SIZE:
    non_travel_sample = non_travel_sms.sample(n=NON_TRAVEL_SAMPLE_SIZE, random_state=42)
    print(f"從 {len(non_travel_sms)} 筆非旅遊簡訊中採樣 {NON_TRAVEL_SAMPLE_SIZE} 筆")
else:
    non_travel_sample = non_travel_sms
    print(f"使用全部 {len(non_travel_sms)} 筆非旅遊簡訊")

non_travel_pairs = find_same_type_pairs(
    df_subset=non_travel_sample,
    max_pairs=4000,  
    description="非旅遊類型簡訊"
)

# ========== 合併結果 ==========
all_similar_pairs = travel_pairs + non_travel_pairs

print(f"\n📊 === 總結果 ===")
print(f"旅遊類型相似對: {len(travel_pairs)} 個")
print(f"非旅遊類型相似對: {len(non_travel_pairs)} 個")
print(f"總相似對: {len(all_similar_pairs)} 個")

開始執行分類配對策略...

🏖️ 第一階段：旅遊類型簡訊配對

=== 旅遊類型簡訊 (批量模式) ===
將批量比較 1000 筆簡訊
目標找到 400 個相似對


旅遊類型簡訊批量比較進度:   0%|          | 0/334 [00:00<?, ?it/s]

旅遊類型簡訊批量比較進度:  60%|██████    | 201/334 [50:11<33:12, 14.98s/it]  



旅遊類型簡訊完成！總共找到 400 個相似對

🏢 第二階段：非旅遊類型簡訊配對
從 208481 筆非旅遊簡訊中採樣 40000 筆

=== 非旅遊類型簡訊 (批量模式) ===
將批量比較 40000 筆簡訊
目標找到 4000 個相似對


非旅遊類型簡訊批量比較進度:  19%|█▉        | 755/4000 [3:23:25<26:50:13, 29.77s/it]

...

非旅遊類型簡訊批量比較進度: 5808it [25:27:24, 34.00s/it] 

Attempt 1 failed: Error code: 429 - {"message": "You have reached the rate limit specific to this model meta-llama/Llama-3.3-70B-Instruct-Turbo-Free. The maximum rate limit for this model is 6.0 queries and 60000 tokens per minute. This limit differs from the general rate limits published at Together AI rate limits documentation (https://docs.together.ai/docs/rate-limits). For inquiries about increasing your model-specific rate limit, please contact our sales team (https://www.together.ai/forms/contact-sales)", "type_": "model_rate_limit"}. Retrying in 60 seconds...


非旅遊類型簡訊批量比較進度: 5813it [25:32:46, 57.29s/it]

Attempt 1 failed: Error code: 503 - The server is overloaded or not ready yet.. Retrying in 60 seconds...


非旅遊類型簡訊批量比較進度: 7113it [32:57:29, 10.09s/it] 

In [7]:
# 保存結果到 CSV（分類版本）
def save_categorized_pairs_to_csv(travel_pairs: List[Tuple[int, int]], 
                                  non_travel_pairs: List[Tuple[int, int]], 
                                  df: pd.DataFrame):
    """保存分類的相似對到 CSV 檔案"""
    
    # 保存旅遊類型相似對
    if travel_pairs:
        travel_df = pd.DataFrame(travel_pairs, columns=["簡訊id", "相似簡訊id"])
        travel_df['類型'] = '旅遊'
        travel_df.to_csv("sms_travel_similar_pairs.csv", index=False, encoding='utf-8-sig')
        print(f"旅遊類型相似對已保存到: sms_travel_similar_pairs.csv ({len(travel_pairs)} 對)")
    
    # 保存非旅遊類型相似對
    if non_travel_pairs:
        non_travel_df = pd.DataFrame(non_travel_pairs, columns=["簡訊id", "相似簡訊id"])
        non_travel_df['類型'] = '非旅遊'
        non_travel_df.to_csv("sms_non_travel_similar_pairs.csv", index=False, encoding='utf-8-sig')
        print(f"非旅遊類型相似對已保存到: sms_non_travel_similar_pairs.csv ({len(non_travel_pairs)} 對)")
    
    # 保存合併結果
    all_pairs = travel_pairs + non_travel_pairs
    if all_pairs:
        # 創建完整的結果 DataFrame
        travel_df_full = pd.DataFrame(travel_pairs, columns=["簡訊id", "相似簡訊id"])
        travel_df_full['類型'] = '旅遊'
        
        non_travel_df_full = pd.DataFrame(non_travel_pairs, columns=["簡訊id", "相似簡訊id"])
        non_travel_df_full['類型'] = '非旅遊'
        
        combined_df = pd.concat([travel_df_full, non_travel_df_full], ignore_index=True)
        
        # 添加簡訊內容（可選）
        def get_sms_content(sms_id):
            content = df[df['sms_id'] == sms_id]['sms_body']
            return content.iloc[0] if len(content) > 0 else "未找到"
        
        combined_df['簡訊內容'] = combined_df['簡訊id'].apply(get_sms_content)
        combined_df['相似簡訊內容'] = combined_df['相似簡訊id'].apply(get_sms_content)
        
        # 保存完整結果
        combined_df.to_csv("sms_all_similar_pairs_with_content.csv", index=False, encoding='utf-8-sig')
        print(f"完整結果已保存到: sms_all_similar_pairs_with_content.csv ({len(all_pairs)} 對)")
        
        # 保存簡潔版本（只有ID）
        simple_df = combined_df[['簡訊id', '相似簡訊id', '類型']]
        simple_df.to_csv("sms_all_similar_pairs.csv", index=False, encoding='utf-8-sig')
        print(f"簡潔版結果已保存到: sms_all_similar_pairs.csv")
        
        return combined_df
    
    return None

# 保存結果
if travel_pairs or non_travel_pairs:
    result_df = save_categorized_pairs_to_csv(travel_pairs, non_travel_pairs, df)
    if result_df is not None:
        print("\n結果預覽:")
        print(result_df[['簡訊id', '相似簡訊id', '類型']].head(10))
else:
    print("未找到任何相似對")

旅遊類型相似對已保存到: sms_travel_similar_pairs.csv (400 對)
非旅遊類型相似對已保存到: sms_non_travel_similar_pairs.csv (2000 對)
完整結果已保存到: sms_all_similar_pairs_with_content.csv (2400 對)
簡潔版結果已保存到: sms_all_similar_pairs.csv

結果預覽:
     簡訊id  相似簡訊id  類型
0  254284   44913  旅遊
1   73455   30973  旅遊
2   73455  358354  旅遊
3   30973  358354  旅遊
4  134878   13188  旅遊
5  296772   37977  旅遊
6  296772  384483  旅遊
7   37977  384483  旅遊
8  303927  221770  旅遊
9  248068  411326  旅遊


In [8]:
# 驗證和分析結果（分類版本）
def analyze_categorized_pairs(travel_pairs: List[Tuple[int, int]], 
                             non_travel_pairs: List[Tuple[int, int]], 
                             df: pd.DataFrame):
    """分析分類相似對的質量"""
    
    print(f"=== 分類相似對分析結果 ===")
    print(f"旅遊類型相似對: {len(travel_pairs)} 個")
    print(f"非旅遊類型相似對: {len(non_travel_pairs)} 個")
    print(f"總相似對: {len(travel_pairs) + len(non_travel_pairs)} 個")
    
    # 分析旅遊類型相似對
    if travel_pairs:
        print(f"\n🏖️ === 旅遊類型相似對樣本 ===")
        import random
        sample_size = min(3, len(travel_pairs))
        sample_pairs = random.sample(travel_pairs, sample_size)
        
        for i, (id1, id2) in enumerate(sample_pairs, 1):
            sms1 = df[df['sms_id'] == id1]['sms_body'].iloc[0]
            sms2 = df[df['sms_id'] == id2]['sms_body'].iloc[0]
            
            print(f"\n--- 旅遊相似對 {i} ---")
            print(f"ID {id1}: {sms1}")
            print(f"ID {id2}: {sms2}")
            print("-" * 50)
    
    # 分析非旅遊類型相似對
    if non_travel_pairs:
        print(f"\n🏢 === 非旅遊類型相似對樣本 ===")
        import random
        sample_size = min(3, len(non_travel_pairs))
        sample_pairs = random.sample(non_travel_pairs, sample_size)
        
        for i, (id1, id2) in enumerate(sample_pairs, 1):
            sms1 = df[df['sms_id'] == id1]['sms_body'].iloc[0]
            sms2 = df[df['sms_id'] == id2]['sms_body'].iloc[0]
            
            print(f"\n--- 非旅遊相似對 {i} ---")
            print(f"ID {id1}: {sms1}")
            print(f"ID {id2}: {sms2}")
            print("-" * 50)
    
    # 統計分析
    if travel_pairs or non_travel_pairs:
        print(f"\n📈 === 統計分析 ===")
        
        # 旅遊類型覆蓋率
        if travel_pairs:
            travel_ids_in_pairs = set()
            for id1, id2 in travel_pairs:
                travel_ids_in_pairs.add(id1)
                travel_ids_in_pairs.add(id2)
            travel_coverage = len(travel_ids_in_pairs) / len(travel_sms) * 100
            print(f"旅遊簡訊覆蓋率: {travel_coverage:.1f}% ({len(travel_ids_in_pairs)}/{len(travel_sms)})")
        
        # 非旅遊類型覆蓋率
        if non_travel_pairs:
            non_travel_ids_in_pairs = set()
            for id1, id2 in non_travel_pairs:
                non_travel_ids_in_pairs.add(id1)
                non_travel_ids_in_pairs.add(id2)
            non_travel_coverage = len(non_travel_ids_in_pairs) / len(non_travel_sample) * 100
            print(f"非旅遊簡訊覆蓋率: {non_travel_coverage:.1f}% ({len(non_travel_ids_in_pairs)}/{len(non_travel_sample)})")

# 分析結果
if travel_pairs or non_travel_pairs:
    analyze_categorized_pairs(travel_pairs, non_travel_pairs, df)

=== 分類相似對分析結果 ===
旅遊類型相似對: 400 個
非旅遊類型相似對: 2000 個
總相似對: 2400 個

🏖️ === 旅遊類型相似對樣本 ===

--- 旅遊相似對 1 ---
ID 37977: 您的出國旅遊計畫已經完成安排，所有的細節都已經確認！機票和住宿已經預訂成功，接送服務也已經安排好。請在出發前再次檢查您的護照有效期，並確認所有旅行資料。如果有任何問題或需求，隨時聯絡我們，我們將全程為您提供協助，祝您旅途愉快！
ID 384483: 您的出國旅程已順利安排，感謝您選擇星際旅行社。我們已為您預訂好機票與住宿，請記得提前抵達機場辦理登機手續，並準備好護照及其他旅行文件。
--------------------------------------------------

--- 旅遊相似對 2 ---
ID 333656: 親愛的顧客，您的西班牙假期已經安排妥當。請確保在出發前準備好所有必要的文件，並提前到達機場辦理登機手續。如果您需要任何幫助，隨時聯絡我們的客服，祝您在西班牙度過一段美好的時光。
ID 60433: 奇遇旅行社通知，您的澳大利亞悉尼與大堡礁之旅已經完成安排，您將於2024年7月25日出發，行程包括遊覽悉尼歌劇院、海港大橋，並前往大堡礁體驗浮潛。請攜帶舒適的鞋子和泳衣，為您準備一場難忘的海洋之旅。
--------------------------------------------------

--- 旅遊相似對 3 ---
ID 55667: 您的悉尼之旅已確認，出發時間為2024年8月5日，航班編號CX-102，請準時登機，祝您有個愉快的假期！
ID 370056: 【旅天下】感謝您選擇我們的服務！您的機票與住宿已經確認。請注意出發前的準備事項，如有任何問題，請隨時聯絡我們的客服部門，我們會提供全方位協助。
--------------------------------------------------

🏢 === 非旅遊類型相似對樣本 ===

--- 非旅遊相似對 1 ---
ID 167818: 感謝您使用來福銀行的線上支付服務，您的付款已順利完成。若有其他付款或帳務問題，請不吝與我們的客服團隊聯繫，我們將竭誠為您服務。
ID 409228: 您好，感謝您在星光購物網的訂購！您的包裹已順利出貨，

## negative gen

## 負樣本生成

使用 LLM 批量評估（5選2）方式生成高質量負樣本。

### 🔧 參數控制

- **負樣本數量**：可以在下面的 cell 中調整 `NEGATIVE_PAIRS_COUNT` 變數
- **批量策略**：每5個候選對選出最不匹配的2個
- **配對策略**：60% 跨類型 + 40% 同類型隨機配對
- **質量保證**：LLM 評估確保負樣本真正不匹配

### 💡 建議數量

- **小規模測試**：50-100 個負樣本
- **中等規模**：500-1000 個負樣本  
- **大規模訓練**：1000+ 個負樣本

根據您的正樣本數量，建議負樣本數量為正樣本的 0.5-2 倍。

In [5]:
# 負樣本數量配置 - 快速選擇
print("=== 負樣本數量配置選項 ===")

# 🔧 選擇一個預設配置，或自定義數量
config_options = {
    "測試": 50,
    "小規模": 200, 
    "中規模": 500,
    "大規模": 1000,
    "超大規模": 2000
}

print("預設配置選項:")
for name, count in config_options.items():
    print(f"  {name}: {count} 個負樣本")

# 🎯 在這裡選擇您想要的配置
#SELECTED_CONFIG = "中規模"  # 📝 修改這裡選擇不同配置
# 或者直接設定自定義數量
CUSTOM_NEGATIVE_COUNT = 1200  # 📝 取消註釋並設定自定義數量

# 應用配置
if 'CUSTOM_NEGATIVE_COUNT' in locals():
    NEGATIVE_PAIRS_COUNT = CUSTOM_NEGATIVE_COUNT
    print(f"\n✅ 使用自定義配置: {NEGATIVE_PAIRS_COUNT} 個負樣本")
elif SELECTED_CONFIG in config_options:
    NEGATIVE_PAIRS_COUNT = config_options[SELECTED_CONFIG]
    print(f"\n✅ 使用 '{SELECTED_CONFIG}' 配置: {NEGATIVE_PAIRS_COUNT} 個負樣本")
else:
    NEGATIVE_PAIRS_COUNT = 500  # 預設值
    print(f"\n⚠️  配置無效，使用預設值: {NEGATIVE_PAIRS_COUNT} 個負樣本")

# 根據現有正樣本數量給出建議
if 'travel_pairs' in locals() and 'non_travel_pairs' in locals():
    total_positive = len(travel_pairs) + len(non_travel_pairs)
    ratio = NEGATIVE_PAIRS_COUNT / total_positive if total_positive > 0 else 0
    
    print(f"\n📊 === 樣本平衡分析 ===")
    print(f"正樣本總數: {total_positive}")
    print(f"預計負樣本: {NEGATIVE_PAIRS_COUNT}")
    print(f"負正比例: {ratio:.2f} (負樣本/正樣本)")
    
    if ratio < 0.5:
        print("💡 建議: 負樣本較少，可能需要增加以平衡數據集")
    elif ratio > 2.0:
        print("💡 建議: 負樣本較多，可能影響訓練效率")
    else:
        print("✅ 負正樣本比例合理")
else:
    print("\n💡 提示: 請先執行正樣本生成以獲得更準確的建議")

print(f"\n🚀 準備生成 {NEGATIVE_PAIRS_COUNT} 個負樣本...")

=== 負樣本數量配置選項 ===
預設配置選項:
  測試: 50 個負樣本
  小規模: 200 個負樣本
  中規模: 500 個負樣本
  大規模: 1000 個負樣本
  超大規模: 2000 個負樣本

✅ 使用自定義配置: 1200 個負樣本

💡 提示: 請先執行正樣本生成以獲得更準確的建議

🚀 準備生成 1200 個負樣本...


In [6]:
RPM = 60                # requests/min
WINDOW = 60             # sec
MAX_RETRY = 3           # retries
MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

# 請求紀錄
_req_log = []

def _throttle():
    """簡單的限流機制"""
    now = time.time()
    # 清理超過時間窗口的記錄
    _req_log[:] = [t for t in _req_log if now - t < WINDOW]
    
    # 檢查是否需要等待
    if len(_req_log) >= RPM:
        wait_time = WINDOW - (now - _req_log[0])
        if wait_time > 0:
            time.sleep(wait_time)
    
    _req_log.append(now)

# 負樣本生成 - 找出最不匹配的簡訊對
def generate_negative_pairs_batch(df: pd.DataFrame, max_negative_pairs: int = 1000) -> List[Tuple[int, int]]:
    """
    生成負樣本：找出最不匹配的簡訊對
    使用批量比較，一次給模型 5 個選擇進行評估
    
    策略：
    1. 跨類型配對（旅遊 vs 非旅遊）
    2. 同類型但主題差異大的配對
    3. 批量評估 5 個候選對，選出最不匹配的
    
    Args:
        df: 完整的簡訊 DataFrame
        max_negative_pairs: 最大負樣本對數量
        
    Returns:
        負樣本對的列表 [(id1, id2), ...]
    """
    
    negative_pairs = []
    
    # 分離不同類型的簡訊
    travel_df = df[df['label'] == 1].copy()
    non_travel_df = df[df['label'] != 1].copy()
    
    print(f"=== 負樣本生成策略 ===")
    print(f"旅遊類型簡訊: {len(travel_df)} 筆")
    print(f"非旅遊類型簡訊: {len(non_travel_df)} 筆")
    print(f"目標生成負樣本對: {max_negative_pairs} 個")
    
    # 負樣本評估 prompt
    negative_evaluation_prompt = textwrap.dedent(
        """
        你是一個專業的文本相似性判斷助手。現在有 5 組簡訊對，請評估每組的相似性，並選出其中最不相似的 2 組。

        評估標準：
        1. 主題差異：主題越不同，負樣本質量越好
        2. 意圖差異：目的越不同，負樣本質量越好  
        3. 內容差異：內容越無關，負樣本質量越好
        4. 語境差異：使用場景越不同，負樣本質量越好

        請仔細比較以下 5 組簡訊對：

        組A:
        簡訊1: {sms_a1}
        簡訊2: {sms_a2}

        組B:
        簡訊1: {sms_b1}
        簡訊2: {sms_b2}

        組C:
        簡訊1: {sms_c1}
        簡訊2: {sms_c2}

        組D:
        簡訊1: {sms_d1}
        簡訊2: {sms_d2}

        組E:
        簡訊1: {sms_e1}
        簡訊2: {sms_e2}

        請按照以下格式回答，選出最不相似的 2 組：
        最不相似組1: [組別字母]
        最不相似組2: [組別字母]
        """
    )
    
    def evaluate_negative_batch(candidate_pairs: List[Tuple[Tuple[int, str], Tuple[int, str]]]) -> List[Tuple[int, int]]:
        """
        評估一批候選負樣本對，返回最不匹配的 2 個
        
        Args:
            candidate_pairs: [((id1, content1), (id2, content2)), ...] 最多 5 個候選對
            
        Returns:
            最不匹配的 2 個對 [(id1, id2), ...]
        """
        if len(candidate_pairs) != 5:
            return []
        
        # 準備 prompt 參數
        prompt_params = {}
        for i, ((id1, content1), (id2, content2)) in enumerate(candidate_pairs):
            group_letter = chr(ord('a') + i)  # a, b, c, d, e
            prompt_params[f'sms_{group_letter}1'] = content1
            prompt_params[f'sms_{group_letter}2'] = content2
        
        prompt = negative_evaluation_prompt.format(**prompt_params)
        
        client = get_current_together_client()
        
        for attempt in range(MAX_RETRY):
            try:
                _throttle()
                response = client.chat.completions.create(
                    model=MODEL_ID,
                    messages=[
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=50,
                    temperature=0.0,
                    stream=False,
                )
                
                result = response.choices[0].message.content.strip()
                
                # 強化版解析邏輯
                selected_pairs = []
                lines = result.split('\n')
                
                for line in lines:
                    if '最不相似組' in line and ':' in line:
                        try:
                            # 提取冒號後的內容
                            content_after_colon = line.split(':')[1].strip()
                            
                            # 查找組別字母 - 支持多種格式
                            group_letter = None
                            
                            # 方法1: 查找 "組X" 格式
                            import re
                            group_match = re.search(r'組([A-Ea-e])', content_after_colon)
                            if group_match:
                                group_letter = group_match.group(1).lower()
                            
                            # 方法2: 查找單獨的字母
                            if not group_letter:
                                letter_match = re.search(r'\b([A-Ea-e])\b', content_after_colon)
                                if letter_match:
                                    group_letter = letter_match.group(1).lower()
                            
                            # 方法3: 直接查找 a-e 字母
                            if not group_letter:
                                for letter in ['a', 'b', 'c', 'd', 'e']:
                                    if letter in content_after_colon.lower():
                                        group_letter = letter
                                        break
                            
                            if group_letter and group_letter in ['a', 'b', 'c', 'd', 'e']:
                                group_index = ord(group_letter) - ord('a')
                                if group_index < len(candidate_pairs):
                                    pair = candidate_pairs[group_index]
                                    selected_pair = (pair[0][0], pair[1][0])  # (id1, id2)
                                    selected_pairs.append(selected_pair)
    
                            else:
                                print(f"無法識別組別: '{content_after_colon}'")
                                
                        except Exception as parse_error:
                            print(f"解析錯誤: {parse_error}")
                            continue
            
                
                # 確保最多返回 2 個對，並去重
                unique_pairs = []
                for pair in selected_pairs:
                    if pair not in unique_pairs:
                        unique_pairs.append(pair)
                
                return unique_pairs[:2]
                
            except Exception as e:
                if attempt + 1 >= MAX_RETRY:
                    print(f"負樣本評估失敗: {e}")
                    return []
                
                print(f"嘗試 {attempt + 1} 失敗: {e}. 60秒後重試...")
                time.sleep(60)
        
        return []
    
    # 生成候選負樣本對
    def generate_candidate_pairs(travel_df: pd.DataFrame, non_travel_df: pd.DataFrame, num_candidates: int) -> List[Tuple[Tuple[int, str], Tuple[int, str]]]:
        """生成候選負樣本對"""
        candidates = []
        
        # 策略1: 跨類型配對 (60%)
        cross_type_count = int(num_candidates * 0.6)
        for _ in range(cross_type_count):
            if len(travel_df) > 0 and len(non_travel_df) > 0:
                travel_sample = travel_df.sample(n=1).iloc[0]
                non_travel_sample = non_travel_df.sample(n=1).iloc[0]
                
                candidates.append((
                    (travel_sample['sms_id'], travel_sample['sms_body']),
                    (non_travel_sample['sms_id'], non_travel_sample['sms_body'])
                ))
        
        # 策略2: 同類型隨機配對 (40%)
        remaining_count = num_candidates - len(candidates)
        
        # 旅遊類型內隨機配對
        travel_internal_count = remaining_count // 2
        if len(travel_df) >= 2:
            for _ in range(travel_internal_count):
                samples = travel_df.sample(n=2)
                sample1, sample2 = samples.iloc[0], samples.iloc[1]
                
                candidates.append((
                    (sample1['sms_id'], sample1['sms_body']),
                    (sample2['sms_id'], sample2['sms_body'])
                ))
        
        # 非旅遊類型內隨機配對
        non_travel_internal_count = num_candidates - len(candidates)
        if len(non_travel_df) >= 2:
            for _ in range(non_travel_internal_count):
                samples = non_travel_df.sample(n=2)
                sample1, sample2 = samples.iloc[0], samples.iloc[1]
                
                candidates.append((
                    (sample1['sms_id'], sample1['sms_body']),
                    (sample2['sms_id'], sample2['sms_body'])
                ))
        
        return candidates
    
    # 主要處理邏輯
    total_batches = (max_negative_pairs + 1) // 2  # 每批最多生成 2 個負樣本
    
    print(f"\n開始生成負樣本，預計需要 {total_batches} 批次處理")
    
    with tqdm(total=max_negative_pairs, desc="負樣本生成進度") as pbar:
        for batch_idx in range(total_batches):
            if len(negative_pairs) >= max_negative_pairs:
                break
            
            # 生成 5 個候選對
            candidates = generate_candidate_pairs(travel_df, non_travel_df, 5)
            if len(candidates) == 5:
                # 評估並選出最不匹配的 2 個
                selected_negative_pairs = evaluate_negative_batch(candidates)
                for pair in selected_negative_pairs:
                    if len(negative_pairs) < max_negative_pairs:
                        negative_pairs.append(pair)
                        pbar.update(1)
                        
                        # 顯示找到的負樣本
                        id1, id2 = pair
                        sms1 = df[df['sms_id'] == id1]['sms_body'].iloc[0]
                        sms2 = df[df['sms_id'] == id2]['sms_body'].iloc[0]
                        
            else:
                print(f"警告: 批次 {batch_idx + 1} 候選對不足 5 個")
    
    print(f"\n負樣本生成完成！總共生成 {len(negative_pairs)} 個負樣本對")
    return negative_pairs

# 執行負樣本生成 - 使用配置的數量
print("開始生成高質量負樣本...")

# 檢查是否有預設的數量配置
if 'NEGATIVE_PAIRS_COUNT' not in locals():
    NEGATIVE_PAIRS_COUNT = 500  # 預設值
    print(f"⚠️  未找到配置，使用預設值: {NEGATIVE_PAIRS_COUNT} 個負樣本")

print(f"🎯 目標生成: {NEGATIVE_PAIRS_COUNT} 個負樣本對")

# 估算所需時間
estimated_batches = (NEGATIVE_PAIRS_COUNT + 1) // 2
estimated_api_calls = estimated_batches
estimated_minutes = estimated_api_calls / (RPM / 4)  # 考慮4個API key的並行
print(f"📅 預估處理時間: {estimated_minutes:.1f} 分鐘 ({estimated_batches} 批次, {estimated_api_calls} API調用)")

# 執行生成
negative_pairs = generate_negative_pairs_batch(df, max_negative_pairs=NEGATIVE_PAIRS_COUNT)

print(f"\n📊 === 負樣本生成結果 ===")
print(f"✅ 成功生成: {len(negative_pairs)} 個負樣本對")
print(f"🎯 目標數量: {NEGATIVE_PAIRS_COUNT}")
print(f"📈 完成率: {len(negative_pairs)/NEGATIVE_PAIRS_COUNT*100:.1f}%")

if len(negative_pairs) < NEGATIVE_PAIRS_COUNT:
    shortfall = NEGATIVE_PAIRS_COUNT - len(negative_pairs)
    print(f"⚠️  短缺 {shortfall} 個負樣本，可能是由於API限制或數據限制")
    print("💡 建議: 可以重新執行此cell或調整批次大小")

# 分析負樣本質量
def analyze_negative_pairs(negative_pairs: List[Tuple[int, int]], df: pd.DataFrame):
    """分析負樣本的質量"""
    
    print(f"\n=== 負樣本質量分析 ===")
    
    cross_type_count = 0
    same_type_count = 0
    
    for id1, id2 in negative_pairs:
        label1 = df[df['sms_id'] == id1]['label'].iloc[0]
        label2 = df[df['sms_id'] == id2]['label'].iloc[0]
        
        if label1 != label2:
            cross_type_count += 1
        else:
            same_type_count += 1
    
    print(f"跨類型負樣本: {cross_type_count} 個 ({cross_type_count/len(negative_pairs)*100:.1f}%)")
    print(f"同類型負樣本: {same_type_count} 個 ({same_type_count/len(negative_pairs)*100:.1f}%)")
    
    # 顯示一些負樣本例子
    print(f"\n=== 負樣本例子 ===")
    import random
    sample_size = min(3, len(negative_pairs))
    sample_pairs = random.sample(negative_pairs, sample_size)
    
    for i, (id1, id2) in enumerate(sample_pairs, 1):
        sms1 = df[df['sms_id'] == id1]['sms_body'].iloc[0]
        sms2 = df[df['sms_id'] == id2]['sms_body'].iloc[0]
        label1 = df[df['sms_id'] == id1]['label'].iloc[0]
        label2 = df[df['sms_id'] == id2]['label'].iloc[0]
        
        print(f"\n--- 負樣本 {i} ---")
        print(f"ID {id1} (label={label1}): {sms1}")
        print(f"ID {id2} (label={label2}): {sms2}")
        print("-" * 60)

if negative_pairs:
    analyze_negative_pairs(negative_pairs, df)

開始生成高質量負樣本...
🎯 目標生成: 1200 個負樣本對
📅 預估處理時間: 40.0 分鐘 (600 批次, 600 API調用)
=== 負樣本生成策略 ===
旅遊類型簡訊: 1000 筆
非旅遊類型簡訊: 208481 筆
目標生成負樣本對: 1200 個

開始生成負樣本，預計需要 600 批次處理


負樣本生成進度:  99%|█████████▉| 1192/1200 [1:04:45<00:26,  3.26s/it]



負樣本生成完成！總共生成 1192 個負樣本對

📊 === 負樣本生成結果 ===
✅ 成功生成: 1192 個負樣本對
🎯 目標數量: 1200
📈 完成率: 99.3%
⚠️  短缺 8 個負樣本，可能是由於API限制或數據限制
💡 建議: 可以重新執行此cell或調整批次大小

=== 負樣本質量分析 ===
跨類型負樣本: 1183 個 (99.2%)
同類型負樣本: 9 個 (0.8%)

=== 負樣本例子 ===

--- 負樣本 1 ---
ID 418547 (label=nan): 您的越南胡志明市與美奈海灘之旅已確定！由世界旅遊公司安排的這次行程將帶您遊覽胡志明市的歷史景點，並在美奈的沙灘上放鬆。行程中還安排了越南的特色美食與當地文化體驗，讓您充分感受越南的熱情。
ID 248141 (label=nan): 您好，這是來自中信銀行的提醒。您的信用卡款項至今尚未繳納，並已經過了繳款期限。為了避免影響您的信用記錄及產生滯納金，請儘速繳納所欠款項。如果已經繳納，請忽略此簡訊。若有任何問題，歡迎隨時來電詢問。
------------------------------------------------------------

--- 負樣本 2 ---
ID 359686 (label=1.0): 【巴黎自由行】限時優惠，讓你在浪漫之都輕鬆享受無憂旅遊！行程安排精緻，現正開放報名中，快來參加！https://paristravel.com
ID 128624 (label=nan): 您好，您於中信銀行申請之卡友貸款至今尚欠NT$15,700，逾期已產生違約利息，若今日未清償將由第三方催收公司處理，請儘速處理。
------------------------------------------------------------

--- 負樣本 3 ---
ID 243587 (label=1.0): 莊哲銘推薦使用「旅圈筆記APP」製作個人旅遊規劃表，可共享行程給同行者，功能包含換匯提醒、天氣預報、票券提醒！
ID 12142 (label=nan): 台大醫院健康管理中心特別推出秋冬季護肝健檢專案，內容包含肝功能抽血、腹部超音波與專業醫師解說報告，預約即贈肝臟保健小手冊。
------------------------------

In [7]:
# 保存負樣本結果
def save_negative_pairs_to_csv(negative_pairs: List[Tuple[int, int]], df: pd.DataFrame):
    """保存負樣本對到 CSV 檔案"""
    
    if not negative_pairs:
        print("沒有負樣本可保存")
        return None
    
    # 創建負樣本 DataFrame
    negative_df = pd.DataFrame(negative_pairs, columns=["簡訊id", "相似簡訊id"])
    negative_df['類型'] = '負樣本'
    
    # 添加簡訊內容和標籤
    def get_sms_info(sms_id):
        row = df[df['sms_id'] == sms_id]
        if len(row) > 0:
            return row.iloc[0]['sms_body'], row.iloc[0]['label']
        return "未找到", -1
    
    # 獲取簡訊內容和標籤
    sms1_info = negative_df['簡訊id'].apply(get_sms_info)
    sms2_info = negative_df['相似簡訊id'].apply(get_sms_info)
    
    negative_df['簡訊內容'] = [info[0] for info in sms1_info]
    negative_df['簡訊標籤'] = [info[1] for info in sms1_info]
    negative_df['相似簡訊內容'] = [info[0] for info in sms2_info]
    negative_df['相似簡訊標籤'] = [info[1] for info in sms2_info]
    
    # 添加配對類型分析
    def get_pair_type(label1, label2):
        if label1 == label2:
            return '同類型'
        else:
            return '跨類型'
    
    negative_df['配對類型'] = negative_df.apply(
        lambda row: get_pair_type(row['簡訊標籤'], row['相似簡訊標籤']), axis=1
    )
    
    # 保存完整版本
    negative_df.to_csv("sms_negative_pairs_with_content.csv", index=False, encoding='utf-8-sig')
    print(f"完整負樣本結果已保存到: sms_negative_pairs_with_content.csv ({len(negative_pairs)} 對)")
    
    # 保存簡潔版本（只有ID和類型）
    simple_negative_df = negative_df[['簡訊id', '相似簡訊id', '類型', '配對類型']]
    simple_negative_df.to_csv("sms_negative_pairs.csv", index=False, encoding='utf-8-sig')
    print(f"簡潔負樣本結果已保存到: sms_negative_pairs.csv")
    
    return negative_df

# 合併正負樣本結果
def create_complete_training_dataset(travel_pairs: List[Tuple[int, int]], 
                                   non_travel_pairs: List[Tuple[int, int]],
                                   negative_pairs: List[Tuple[int, int]], 
                                   df: pd.DataFrame):
    """創建完整的訓練數據集（正樣本 + 負樣本）"""
    
    all_pairs = []
    
    # 添加旅遊正樣本
    for pair in travel_pairs:
        all_pairs.append({
            '簡訊id': pair[0],
            '相似簡訊id': pair[1],
            '標籤': 1,  # 正樣本
            '樣本類型': '旅遊正樣本'
        })
    
    # 添加非旅遊正樣本
    for pair in non_travel_pairs:
        all_pairs.append({
            '簡訊id': pair[0],
            '相似簡訊id': pair[1],
            '標籤': 1,  # 正樣本
            '樣本類型': '非旅遊正樣本'
        })
    
    # 添加負樣本
    for pair in negative_pairs:
        all_pairs.append({
            '簡訊id': pair[0],
            '相似簡訊id': pair[1],
            '標籤': 0,  # 負樣本
            '樣本類型': '負樣本'
        })
    
    # 創建完整數據集
    complete_df = pd.DataFrame(all_pairs)
    
    # 添加簡訊內容
    def get_sms_content(sms_id):
        content = df[df['sms_id'] == sms_id]['sms_body']
        return content.iloc[0] if len(content) > 0 else "未找到"
    
    complete_df['簡訊內容'] = complete_df['簡訊id'].apply(get_sms_content)
    complete_df['相似簡訊內容'] = complete_df['相似簡訊id'].apply(get_sms_content)
    
    # 保存完整訓練數據集
    complete_df.to_csv("sms_complete_training_dataset.csv", index=False, encoding='utf-8-sig')
    
    # 統計信息
    print(f"\n📊 === 完整訓練數據集統計 ===")
    print(f"旅遊正樣本: {len(travel_pairs)} 對")
    print(f"非旅遊正樣本: {len(non_travel_pairs)} 對")
    print(f"負樣本: {len(negative_pairs)} 對")
    print(f"總樣本: {len(all_pairs)} 對")
    
    positive_ratio = (len(travel_pairs) + len(non_travel_pairs)) / len(all_pairs) * 100
    negative_ratio = len(negative_pairs) / len(all_pairs) * 100
    
    print(f"正樣本比例: {positive_ratio:.1f}%")
    print(f"負樣本比例: {negative_ratio:.1f}%")
    print(f"完整訓練數據集已保存到: sms_complete_training_dataset.csv")
    
    return complete_df

# 保存負樣本結果
if negative_pairs:
    negative_result_df = save_negative_pairs_to_csv(negative_pairs, df)
    
    # 顯示負樣本結果預覽
    if negative_result_df is not None:
        print(f"\n=== 負樣本結果預覽 ===")
        print(negative_result_df[['簡訊id', '相似簡訊id', '配對類型']].head(10))
        
        print("\n✅ 負樣本生成和保存完成！")
        print("📁 已保存檔案:")
        print("  - sms_negative_pairs.csv (簡潔版)")
        print("  - sms_negative_pairs_with_content.csv (完整版)")
else:
    print("沒有生成負樣本，請先執行負樣本生成")

完整負樣本結果已保存到: sms_negative_pairs_with_content.csv (1192 對)
簡潔負樣本結果已保存到: sms_negative_pairs.csv

=== 負樣本結果預覽 ===
     簡訊id  相似簡訊id 配對類型
0   19635  267771  跨類型
1  329175   15069  跨類型
2  291076  393114  跨類型
3  255383   75606  跨類型
4   41187  391525  跨類型
5  282119  220849  跨類型
6  137836  134915  跨類型
7  185111  259560  跨類型
8  272034  342824  跨類型
9  319287  402629  跨類型

✅ 負樣本生成和保存完成！
📁 已保存檔案:
  - sms_negative_pairs.csv (簡潔版)
  - sms_negative_pairs_with_content.csv (完整版)
